In [1]:
from datasets import load_from_disk
import datasets
from nlplog import Config, FailurePredModel, FailurePredDataset
from trainer import seed_everything, train_and_eval, test
import pandas as pd
from datasets import concatenate_datasets
import copy
import torch


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /data/user/nyf/pkgs/anaconda3/envs/loggpt did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"zh-cn"},"_languagePackId"'), PosixPath('"zh-cn","availableLanguages"'), PosixPath('"/data/user/nyf/.vscode-server/data/clp/0860ebc1a1c97b3d555b187ac30e07fe.zh-cn","_resolvedLanguagePackCoreLocation"'), PosixPath('

In [2]:
train_ds = load_from_disk('../data_hf/BGL/single/train_3611')
test_20000_ds = load_from_disk('../data_hf/BGL/single/test_20000')
test_302_all_ds = load_from_disk('../data_hf/BGL/single/all_302')

In [3]:
test_302_all_ds.to_pandas().labels.value_counts()

labels
0    256
1     46
Name: count, dtype: int64

In [34]:
CFG_failure_pred = Config(
    wandb_project='failure_prediction',
    wandb_enable=False,
    single_linear_hidden_size=64,
    epochs=100,
    batch_size=16,
    train_ratio=0.8,
    lr=1e-4,
    lr_scheduler='none',
    warmup_steps=0,
    early_stop=10,
    weight_metric=False,
    tqdm_disable=True,
    confusion_matrix_enable=False,
    train_data_file='../data/bgl/train80_bgl_structured_unique.csv',
    test_data_file='../data/bgl/test20_bgl_structured_unique.csv',
    random_state=1006
)

In [35]:
train_ds_shuffle = train_ds.shuffle(seed=CFG_failure_pred.random_state)
normal_train_ds = train_ds_shuffle.filter(lambda example: example['labels'] == 0)
abnormal_train_ds = train_ds_shuffle.filter(lambda example: example['labels'] == 1)

In [51]:
# 16 shots demo
CFG_failure_pred.lr = 1e-4
CFG_failure_pred.epochs = 200
CFG_failure_pred.early_stop = 20
CFG_failure_pred.best_model_metric = 'binary'
CFG_failure_pred.single_linear_hidden_size = 256
CFG_failure_pred.device = torch.device('cuda:0')
CFG_failure_pred.batch_size = 16
CFG_failure_pred.train_ratio = 1

few_shots = concatenate_datasets([normal_train_ds.select(range(16//2)), abnormal_train_ds.select(range(16//2))])
few_shots = few_shots.shuffle(seed=CFG_failure_pred.random_state)

seed_everything(CFG_failure_pred.random_state)
llm_hidden_size = 4096
task_module = FailurePredModel(llm_hidden_size=llm_hidden_size, config=CFG_failure_pred)

CFG_failure_pred.confusion_matrix_enable = False

CFG_failure_pred.f1_report = ['binary']
train_and_eval(task_module, FailurePredDataset, few_shots, CFG_failure_pred)

CFG_failure_pred.confusion_matrix_enable = True
CFG_failure_pred.f1_report = ['binary', 'micro', 'macro']

# 20000 test
report = test(task_module, FailurePredDataset, test_20000_ds, CFG_failure_pred, load_best_model=False)

load dataset
load dataloader
/data/user/nyf/pkgs/anaconda3/envs/loggpt/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Save best binary-f1 score: 0.6667 FailurePredModel
Save best binary-f1 score: 0.6957 FailurePredModel
Save best binary-f1 score: 0.7619 FailurePredModel
Save best binary-f1 score: 0.9412 FailurePredModel
Save best binary-f1 score: 1.0000 FailurePredModel


early stop and exit.
================Test================
confusion_matrix:
[[12685  2421]
 [  236  4658]]
[binary] precision: 0.6580 recall: 0.9518 f1: 0.7781
[micro] precision: 0.8671 recall: 0.8671 f1: 0.8672
[macro] precision: 0.8199 recall: 0.8958 f1: 0.8416
